In [15]:
import sys
from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import (QSystemTrayIcon, QApplication, QLabel, QMainWindow, QPushButton, QWidget, 
                             QVBoxLayout, QTextEdit, QDialog, QComboBox)
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import Qt, QCoreApplication
import tkinter as tk
import PIL.Image
import numpy as np
import cv2
import io
import json
import ctypes
from io import BytesIO
import win32clipboard
import glob
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output, Image, display
from google.cloud.vision import types
from google.cloud import vision
import mss
import pyperclip
import mss.tools
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/poorv/Downloads/ocr_scripts/keys/direct-outlook-270501-c05c2d97d1c6.json"

In [33]:
def send_text_clipboard(text):
    pyperclip.copy(text)
    spam = pyperclip.paste()
    
def dtlh(path, lh):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    if type(path) == str:
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
            image = vision.types.Image(content=content)
    else:
        image = types.Image(content=cv2.imencode('.jpg', path)[1].tostring())
        
    response = client.text_detection(
    image=image,
    image_context={"language_hints": lh},
    )
    texts = response.text_annotations
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    try:
        return texts[0].description
    except IndexError:
        print('no text in image')

In [35]:
dtlh(r"C:\Users\poorv\Downloads\ocr_scripts\demo\pg191_excerpt.png", [])


"ఎటితే\nఎటికే exata\n?\nn. [move] pulling (C. (Kitt.)) [Ka.\nD504(a)].\n? wes eravu\nn. [move] 1 pulling, dragging\n2 spasm, cramp (My. (Kitt.)) [Ka. D504(a)].\nఎటివు' eravu\n2\nw vt. [move] 1 to rub;\nto rub off or out; to smear gently 2 to stroke gently\n[Ka. D505].\n"

In [39]:
class ocrWidget(QtWidgets.QWidget):
#     clicked = QtCore.pyqtSignal(object)
    
    def __init__(self, parent = None, lang_hint = []):
        super().__init__()
        root = tk.Tk()
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.setGeometry(0, 0, screen_width, screen_height)
        self.setWindowTitle(' ')
        self.begin = QtCore.QPoint()
        self.end = QtCore.QPoint()
        self.parent = parent
        self.setWindowOpacity(0.3)
        QtWidgets.QApplication.setOverrideCursor(
            QtGui.QCursor(QtCore.Qt.CrossCursor)
        )
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)
        QtWidgets.QShortcut(
            QtGui.QKeySequence("Escape"), self, activated=self.on_Escape
        )
        self.lang_hint = lang_hint
        print(self.lang_hint)

    def paintEvent(self, event):
        qp = QtGui.QPainter(self)
        qp.setPen(QtGui.QPen(QtGui.QColor('black'), 3))
        qp.setBrush(QtGui.QColor(128, 128, 255, 128))
        qp.drawRect(QtCore.QRect(self.begin, self.end))

    def mousePressEvent(self, event):
        self.begin = event.pos()
        self.end = self.begin
        self.update()
    
    @QtCore.pyqtSlot()
    def on_Escape(self):
        print("main esp exit")
#         self.close()
        self.closeAndReturn()
        

    def mouseMoveEvent(self, event):
        self.end = event.pos()
        self.update()
    
    def closeAndReturn(self):
        self.close()
        self.parent.setWindowOpacity(1.)
        return

    def mouseReleaseEvent(self, event):
        previous = self.parent
        self.close()

        x1 = min(self.begin.x(), self.end.x())
        y1 = min(self.begin.y(), self.end.y())
        x2 = max(self.begin.x(), self.end.x())
        y2 = max(self.begin.y(), self.end.y())
        
        if (y1 == y2 and x1 == x2):
            print("no region selected")
            self.closeAndReturn()
        
        with mss.mss() as sct:
            # The screen part to capture            
            monitor = {"top": y1, "left":x1, "width": abs(x2 - x1) , "height": abs(y2 - y1)}
            output = "sct-{top}x{left}_{width}x{height}.png".format(**monitor)
            sct_img = sct.grab(monitor)
            try:
                img = PIL.Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX")
                img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
                im_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                ret,thresh = cv2.threshold(im_gray,127,255,cv2.THRESH_BINARY_INV)
#                 ocr = dtlh(thresh, ["hi"])
                ocr = dtlh(thresh, self.lang_hint)
                if ocr:
#                     send_text_clipboard(ocr)
                    print(ocr)
                self.parent.setWindowOpacity(1.)
            except ValueError:
                print("invalid region selection. try again")
                self.closeAndReturn()

In [58]:
from PyQt5 import QtGui, QtCore
import sys

lang_hints_dict = {u'Afrikaans - Afrikaans': "af", 
u'Albanian - shqip': "sq", 
u'Arabic - العربية': "ar", 
u'Armenian - Հայ': "hy", 
u'Belorussian - беларускі': "be", 
u'Bulgarian - български': "bg", 
u'Catalan - Català': "ca", 
u'Chinese - 普通话': "zh", 
u'Croatian - Hrvatski': "hr", 
u'Czech - Čeština': "cs", 
u'Danish - Dansk': "da", 
u'Dutch - Nederlands': "nl", 
u'English  - English': "en", 
u'Estonian - Eesti keel': "et", 
u'Filipino - Filipino': "fil (or tl)", 
u'Finnish - Suomi': "fi", 
u'French  - Français': "fr", 
u'German - Deutsch': "de", 
u'Greek - Ελληνικά': "el", 
u'Gujarati - ગુજરાતી': "gu", 
u'Hebrew - עברית': "iw", 
u'Hindi - हिन्दी': "hi", 
u'Hungarian - Magyar': "hu", 
u'Icelandic - Íslenska': "is", 
u'Indonesian - Bahasa Indonesia': "id", 
u'Italian - Italiano': "it", 
u'Japanese - 日本語': "ja", 
u'Kannada - ಕನ್ನಡ': "kn", 
u'Khmer - ភាសាខ្មែរ': "km", 
u'Korean - 한국어': "ko", 
u'Lao - ລາວ': "lo", 
u'Latvian - Latviešu': "lv", 
u'Lithuanian - Lietuvių': "lt", 
u'Macedonian - Македонски': "mk", 
u'Malay - Bahasa Melayu': "ms", 
u'Malayalam - മലയാളം': "ml", 
u'Marathi - मराठी': "mr", 
u'Nepali - नेपाली': "ne", 
u'Norwegian - Norsk': "no", 
u'Persian - فارسی': "fa", 
u'Polish - Polski': "pl", 
u'Portuguese - Português': "pt", 
u'Punjabi - ਪੰਜਾਬੀ': "pa", 
u'Romanian - Română': "ro", 
u'Russian - Русский': "ru", 
u'Russian - Русский (старая орфография)': "ru-PETR1708", 
u'Serbian - Српски': "sr", 
u'Serbian - Српски (латиница)': "sr-Latn", 
u'Slovak - Slovenčina': "sk", 
u'Slovenian - Slovenščina': "sl", 
u'Spanish - Español': "es", 
u'Swedish - Svenska': "sv", 
u'Tamil - தமிழ்': "ta", 
u'Telugu - తెలుగు': "te", 
u'Thai - ไทย': "th", 
u'Turkish - Türkçe': "tr", 
u'Ukrainian - Українська': "uk", 
u'Vietnamese - Tiếng Việt': "vi", 
u'Yiddish - Yiddish': "yi"}

class First(QMainWindow):
    def __init__(self, parent=None):
        super(First, self).__init__(parent)
        self.pushButton = QPushButton("click me")
        self.pushButton.clicked.connect(self.on_pushButton_clicked)
        self.dialogs = list()
        self.combo = QComboBox(self)
        self.populateDict()
        self.combo.activated.connect(self.handleActivated)
        self.lang_hint = []
        layout = QVBoxLayout()
        layout.addWidget(self.combo)
        layout.addWidget(self.pushButton)
        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
    
    def on_pushButton_clicked(self):
        dialog = ocrWidget(self, self.lang_hint)
        self.dialogs.append(dialog)
        dialog.show()
        self.setWindowOpacity(0.)
        
    def handleActivated(self, index):
        self.lang_hint = [self.combo.itemData(index)]
        print(self.combo.itemText(index))
        print(self.lang_hint)
        
    def populateDict(self):
        for key, value in lang_hints_dict.items():
            self.combo.addItem(key, value)
            
    def closeEvent(self):
        
def main():
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    main = First()
    main.show()
    sys.exit(app.exec_())
    
if __name__ == '__main__':
    main()

Macedonian - Македонски
['mk']
Hindi - हिन्दी
['hi']
['hi']
बारिश होने लगी।
बच्चे तालियाँ बजाने लगे ।
माँ मेरा नाम चिल्लाने लगीं।
हमें प्यास लगने लगी।
मुझे महसूस होने लगा कि यह पाठ
तो कभी खत्म नहीं होगा ।

Hungarian - Magyar
['hu']
Kannada - ಕನ್ನಡ
['kn']
['kn']
ಮನೋಹರ್ : ನಾನು ಧಾರವಾಡಕ್ಕೆ
ಹೋಗಬೇಕು, ಬಸ್ ಪ್ರಯಾಣ
ಒಳ್ಳೆಯದೋ, ರೈಲ್ ಪ್ರಯಾಣ
ಒಳ್ಳೆಯದೋ ?

Icelandic - Íslenska
['is']
Japanese - 日本語
['ja']
Italian - Italiano
['it']
Italian - Italiano
['it']
Italian - Italiano
['it']
Kannada - ಕನ್ನಡ
['kn']
Indonesian - Bahasa Indonesia
['id']
['id']
no region selected
invalid region selection. try again


SystemExit: 0

In [ ]:

#somehow remeber the favorite langs selected
#create keyboard shorcuts??
#consider logic for setting multuple lang hints (i.e. checkbox) 
#settings menu to set quick access langs i.e. english, spanish, french 

In [59]:
import inspect

In [64]:
def guisave(self):

  # Save geometry
    self.settings.setValue('size', self.size())
    self.settings.setValue('pos', self.pos())

    for name, obj in inspect.getmembers(ui):
      # if type(obj) is QComboBox:  # this works similar to isinstance, but missed some field... not sure why?
        if isinstance(obj, QComboBox):
            name = obj.objectName()  # get combobox name
            index = obj.currentIndex()  # get current index from combobox
            text = obj.itemText(index)  # get the text for current index
            settings.setValue(name, text)  # save combobox selection to registry
            
        if isinstance(obj, QCheckBox):
            name = obj.objectName()
            state = obj.isChecked()
            settings.setValue(name, state)


#         if isinstance(obj, QLineEdit):
#             name = obj.objectName()
#             value = obj.text()
#             settings.setValue(name, value)  # save ui values, so they can be restored next time


#         if isinstance(obj, QRadioButton):
#             name = obj.objectName()
#             value = obj.isChecked()  # get stored value from registry
#             settings.setValue(name, value)


def guirestore(self):

  # Restore geometry  
    self.resize(self.settings.value('size', QtCore.QSize(500, 500)))
    self.move(self.settings.value('pos', QtCore.QPoint(60, 60)))

    for name, obj in inspect.getmembers(ui):
        if isinstance(obj, QComboBox):
            index = obj.currentIndex()  # get current region from combobox
            # text   = obj.itemText(index)   # get the text for new selected index
            name = obj.objectName()

            value = (settings.value(name))

            if value == "":
                continue

            index = obj.findText(value)  # get the corresponding index for specified string in combobox

            if index == -1:  # add to list if not found
                obj.insertItems(0, [value])
                index = obj.findText(value)
                obj.setCurrentIndex(index)
            else:
                obj.setCurrentIndex(index)  # preselect a combobox value by index
                
        if isinstance(obj, QCheckBox):
            name = obj.objectName()
            value = settings.value(name)  # get stored value from registry
            if value != None:
                obj.setChecked(strtobool(value))  # restore checkbox

#         if isinstance(obj, QLineEdit):
#             name = obj.objectName()
#             value = (settings.value(name).decode('utf-8'))  # get stored value from registry
#             obj.setText(value)  # restore lineEditFile



#         if isinstance(obj, QRadioButton):
#             name = obj.objectName()
#             value = settings.value(name)  # get stored value from registry
#             if value != None:
#                 obj.setChecked(strtobool(value))
                
def settings(self):
    # use a custom location
    return QtCore.QSettings('app.conf', QtCore.QSettings.IniFormat)